In [10]:
#Import libraries

import numpy as np
import os
import matplotlib.pyplot as plt
import pickle

import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
from keras.utils import to_categorical
from keras.utils import multi_gpu_model
from keras.utils import Sequence
from keras.models import load_model
from tensorflow.distribute import MirroredStrategy

In [11]:
#All paths

Path = "/home/ug-ml/felix-ML/DataGenerator2/Data" #Path where training and validation data is
SaveDataPath = "/home/ug-ml/Documents/GitHub_BigFiles/SaveFolder" #Base directory of place you store information of models
CifFlolder = "/home/ug-ml/felix-ML/DataGenerator2/CifFolder"
SaveFolderName = "/Convnet6" #Will create a folder and put in information about the outcome / inputs
ModelName = "/Convnet6.hdf5"


#Many variables

#Model Variables
input_shape = (36, 128, 128)

#Hyper parameters
learning_rate = 0.0005
l2_regularizer = 0.0001
loss = 'categorical_crossentropy'
optimizer = "RMSprop" #Not a variable ONLY used for a note
batch_size = 32
epochs = 20
ShuffleTrainData = True

#Call back variables
TrainingPatience = 20
CheckPointMonitor = 'val_acc'
EarlyStopMonitor = 'val_acc'

#CPU variables
CPUworkers = 16

#Limit range
LatticeRange = [6, 11]


#List the name of the variables you want to save in a file
VariableListName = ["input_shape", 
                   "learning_rate", "l2_regularizer", "loss", "optimizer", "batch_size", "epochs", "ShuffleTrainData",
                   "TrainingPatience", "CheckPointMonitor", "EarlyStopMonitor",
                   "CPUworkers",
                   "LatticeRange"]

#List the variables in the same order as VariableListName
VariableListValues = [input_shape, 
                   learning_rate, l2_regularizer, loss, optimizer, batch_size, epochs, ShuffleTrainData,
                   TrainingPatience, CheckPointMonitor, EarlyStopMonitor,
                    CPUworkers,
                      LatticeRange]




In [12]:
#Early stopping and check points

EarlyStop = EarlyStopping(monitor = EarlyStopMonitor,
                          mode = 'auto',
                          verbose = 1,
                          patience = TrainingPatience)

NewPath = SaveDataPath + SaveFolderName
Checkpoint = ModelCheckpoint(NewPath + ModelName, #Save path
                             monitor = CheckPointMonitor,
                             verbose = 1,
                             save_best_only = True,
                             mode = 'auto',
                             save_freq = 'epoch')


#Make folder to put model and history information
try:
    os.mkdir(NewPath)
except:
    print("Folder failed to be created, it may already exist")
    
File1  = open(NewPath +"/Parameters.txt", "w+")
if(len(VariableListName) == len(VariableListValues)):
    for i in range(0, len(VariableListName)):
        File1.write(VariableListName[i] + " " + str(VariableListValues[i]) + "\n")
    File1.close()
else:
    print("VariableListName and VariableListValues do not match up, so file can not be saved")


Folder failed to be created, it may already exist


In [25]:
#Build model
#strategy = MirroredStrategy() #Allows multiple GPUs

#with strategy.scope():
inputs = keras.Input(shape = (36, 128, 128), name = "img1")
x = layers.SeparableConv2D(128, (3, 3), activation='relu', data_format='channels_first', padding = "same")(inputs)
#x = layers.SeparableConv2D(128, (3, 3), activation='relu', data_format='channels_first', padding = "same")(x)
block_1_output = layers.MaxPooling2D((2, 2), data_format='channels_first')(x)

x = layers.SeparableConv2D(128, (3, 3), activation='relu', data_format='channels_first', padding = "same")(block_1_output)
#x = layers.SeparableConv2D(128, (3, 3), activation='relu', data_format='channels_first', padding = "same")(x)
block_2_output = layers.add([x, block_1_output])

#x = layers.Conv2D(128, (3, 3), activation='relu', data_format='channels_first', padding = "same")(block_2_output)
#x = layers.Conv2D(128, (3, 3), activation='relu', data_format='channels_first', padding = "same")(x)
#block_3_output = layers.add([x, block_2_output])

x = layers.SeparableConv2D(128, (3, 3), activation='relu', data_format='channels_first')(block_2_output)
#x = layers.GlobalAveragePooling2D()(x)
x = layers.MaxPooling2D((2, 2), data_format='channels_first')(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.25)(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(10)(x)







model = keras.Model(inputs, outputs, name = "ResNN")
with open(NewPath + '/summary.txt','w') as fh:
    model.summary(print_fn=lambda x: fh.write(x + '\n'))
    
model.summary()

Model: "ResNN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img1 (InputLayer)               [(None, 36, 128, 128 0                                            
__________________________________________________________________________________________________
separable_conv2d_18 (SeparableC (None, 128, 128, 128 5060        img1[0][0]                       
__________________________________________________________________________________________________
max_pooling2d_9 (MaxPooling2D)  (None, 128, 64, 64)  0           separable_conv2d_18[0][0]        
__________________________________________________________________________________________________
separable_conv2d_19 (SeparableC (None, 128, 64, 64)  17664       max_pooling2d_9[0][0]            
______________________________________________________________________________________________

In [26]:
#Load data generators

class FelixSequence(Sequence):
    def __init__(self, x_set, y_set, batch_size, file_type):
        """Here self.x is a list of paths to file_type files. self.y is a
        corresponding list of labels."""
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.file_type = file_type

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return arrs_from_paths(batch_x, self.file_type), to_categorical(np.array(batch_y),10)

def arrs_from_paths(paths, file_type):
    if file_type == "txt":
        return np.array([np.loadtxt(file_name) for file_name in paths])
    elif file_type == "npy":
        return np.array([np.load(file_name) for file_name in paths]) 
    
    
#Define Data gemerator
rng = np.random.default_rng()

PathOfFile = CifFlolder +"/FilePaths.txt"
with open(PathOfFile) as textFile:
    lines = [line.split() for line in textFile]


training_path = []
training_labels = []
validation_path = []
validation_labels = []
for i in lines:
    PathSplit = i[0].split("/")
    for j in PathSplit:
        if(j == "training"):
            if(float(i[3]) > LatticeRange[0] and float(i[3]) < LatticeRange[1]):
                training_path.append(i[0])
                training_labels.append(int(i[2]))
                break
            
        elif(j == "validation"):
            if(float(i[3]) > LatticeRange[0] and float(i[3]) < LatticeRange[1]):
                validation_path.append(i[0])
                validation_labels.append(int(i[2]))
                break

                
trainsize = len(training_path)
validationsize = len(validation_path)

ShuffleTraining = np.arange(trainsize, dtype = np.int)
ShuffleValidation = np.arange(validationsize, dtype = np.int)

training_path_shuffled = training_path.copy()
training_labels_shuffled = training_labels.copy()
validation_path_shuffled = validation_path.copy()
validation_labels_shuffled = validation_labels.copy()

rng.shuffle(ShuffleTraining)
rng.shuffle(ShuffleValidation)

for i in range(0, trainsize):
    training_path_shuffled[i] = training_path[ShuffleTraining[i]]
    training_labels_shuffled[i] = training_labels[ShuffleTraining[i]]
    #print(training_path_shuffled[i], training_labels_shuffled[i])

for i in range(0, validationsize):
    validation_path_shuffled[i] = validation_path[ShuffleValidation[i]]
    validation_labels_shuffled[i] = validation_labels[ShuffleValidation[i]]
    #print(validation_path_shuffled[i], validation_labels_shuffled[i])
    
print("data done.")
training_seq = FelixSequence(training_path_shuffled, training_labels_shuffled, batch_size, "npy")
print("training_seq done.")
val_images = arrs_from_paths(validation_path_shuffled, "npy")
print("val_images done.")
val_lab = to_categorical(validation_labels_shuffled)
print("val_lab done.")
print("Complete")


data done.
training_seq done.
val_images done.
val_lab done.
Complete


In [27]:
model.compile(loss = loss,
                  optimizer = optimizers.RMSprop(learning_rate = learning_rate),
                  metrics=['acc'])


history = model.fit(training_seq, 
                    epochs=epochs, 
                    validation_data = (val_images, val_lab), 
                    callbacks=[EarlyStop, Checkpoint], 
                    workers = CPUworkers,
                    shuffle = ShuffleTrainData,
                    use_multiprocessing = False)

Epoch 1/20
370/370 [==============================] - ETA: 0s - loss: 8.0529 - acc: 0.0989
Epoch 00001: val_acc improved from -inf to 0.10000, saving model to /home/ug-ml/Documents/GitHub_BigFiles/SaveFolder/Convnet6/Convnet6.hdf5
370/370 [==============================] - 103s 279ms/step - loss: 8.0529 - acc: 0.0989 - val_loss: 8.0590 - val_acc: 0.1000
Epoch 2/20
370/370 [==============================] - ETA: 0s - loss: 8.0618 - acc: 0.1000
Epoch 00002: val_acc did not improve from 0.10000
370/370 [==============================] - 103s 279ms/step - loss: 8.0618 - acc: 0.1000 - val_loss: 8.0590 - val_acc: 0.1000
Epoch 3/20
370/370 [==============================] - ETA: 0s - loss: 8.0618 - acc: 0.1000
Epoch 00003: val_acc did not improve from 0.10000
370/370 [==============================] - 102s 276ms/step - loss: 8.0618 - acc: 0.1000 - val_loss: 8.0590 - val_acc: 0.1000
Epoch 4/20
370/370 [==============================] - ETA: 0s - loss: 8.0618 - acc: 0.1000
Epoch 00004: val_acc 